<a href="https://colab.research.google.com/github/NEHALSMITTAL/END-TO-END-MVP/blob/main/Tables1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("main.csv")


In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")


In [ ]:
date_cols = [
    "opening_date",
    "customer_since",
    "last_purchase",
    "promotion_start_date",
    "end_date",
    "transaction_date"
]

for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")


In [ ]:
numeric_cols = [
    "quantity",
    "line_item_amount",
    "discount_percentage",
    "unit_price",
    "total_loya",
    "points_per_unit_spend",
    "min_spend_threshold",
    "bonus_points"
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")


In [ ]:
original_df = df.copy()


In [ ]:
duplicates = original_df[original_df.duplicated()].copy()
duplicates["drop_reason"] = "duplicate_record"

In [ ]:
missing_mandatory = original_df[
    original_df["customer_id"].isna() |
    original_df["transaction_date"].isna() |
    original_df["line_item_amount"].isna()
].copy()
missing_mandatory["drop_reason"] = "missing_mandatory_fields"

In [ ]:
invalid_quantity = original_df[
    original_df["quantity"] <= 0
].copy()
invalid_quantity["drop_reason"] = "invalid_quantity_value"

In [ ]:
invalid_sales = original_df[
    original_df["line_item_amount"] < 0
].copy()
invalid_sales["drop_reason"] = "negative_sales_value"

In [ ]:
if "opening_date" in original_df.columns:
    invalid_dates = original_df[
        original_df["transaction_date"] < original_df["opening_date"]
    ].copy()
else:
    invalid_dates = pd.DataFrame()

if not invalid_dates.empty:
    invalid_dates["drop_reason"] = "transaction_before_store_opening"

In [ ]:
# ⭐ Combine All Dropped Rows
dropped_rows = pd.concat([
    duplicates,
    missing_mandatory,
    invalid_quantity,
    invalid_sales,
    invalid_dates
], ignore_index=True).drop_duplicates()

In [ ]:
if not dropped_rows.empty:
    dropped_rows["drop_timestamp"] = pd.Timestamp.now()

In [ ]:
df = original_df.copy()


In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df.dropna(subset=[
    "customer_id",
    "transaction_date",
    "line_item_amount"
])


df = df[df["quantity"] > 0]


df = df[df["line_item_amount"] >= 0]


if "opening_date" in df.columns:
    df = df[df["transaction_date"] >= df["opening_date"]]



for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())



categorical_cols = [
    "store_region",
    "product_category",
    "loyalty_status",
    "applicable_category"
]

for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].fillna("unknown")



df = df.sort_values(["customer_id", "transaction_date"])
df = df.reset_index(drop=True)



print("✅ Clean Dataset Shape:", df.shape)
print("🚫 Dropped Rows Shape:", dropped_rows.shape)

if not dropped_rows.empty:
    print("\nDrop Reason Summary:")
    print(dropped_rows["drop_reason"].value_counts())



df.to_csv("cleaned_retail_data.csv", index=False)

dropped_rows.to_csv("dropped_rows.csv", index=False)

✅ Clean Dataset Shape: (22523, 36)
🚫 Dropped Rows Shape: (2653, 38)

Drop Reason Summary:
drop_reason
missing_mandatory_fields    2477
negative_sales_value         124
invalid_quantity_value        52
Name: count, dtype: int64
